In [2]:
from pathlib import Path
from typing import List, Dict, Literal
import datetime, time 
import numpy as np 
import pandas as pd

import pickle
class Logger:

    def __init__(self, path='./log/temp', save_interval:int=10000, overwrite_ok=False):
        """
        a shared logger if all control components 

        path is a directory 

        chucks of daat
        """
        path_obj = Path(path)
        if path_obj.exists():
            assert overwrite_ok
        path_obj.mkdir(parents=True, exist_ok=overwrite_ok)


        self.path = path_obj
        self.save_interval = save_interval

        # states
        self.records = []
        self.chuck_number = 0

    def log(self, idx, key, value:Dict):

        """
        ideally all data should have the same keys

        """

        new_record = dict(idx=idx, key=key, value=value)
        self.records.append(new_record)

        if len(self.records)>=self.save_interval:
            self.save()

    def save(self):
        with open(self.path/f"{self.chuck_number}.pkl", "wb") as fp:
            pickle.dump(self.records, fp, pickle.HIGHEST_PROTOCOL)
        self.records = []
        self.chuck_number += 1


    def load(self, min_idx=0, max_idx=999):
        data = []
        
        for idx in range(min_idx, max_idx+1):
            path = self.path/f"{idx}.pkl"
            if not path.exists(): 
                continue
            
            with open(path, 'rb') as fp:
                data.append(pickle.load(fp))

        return sum(data,  start=[])

    def load_as_df(self, min_idx=0, max_idx=999):

        result = self.load(min_idx, max_idx)
        result = pd.DataFrame(result)
        return result.pivot(index='idx', columns=['key'])
                

In [3]:
import RPi.GPIO as GPIO

class MotorPWMControl:

    def __init__(self, pwm_left: GPIO.PWM, pwm_right: GPIO.PWM, logger: Logger, name='MotorPWMControl'):
        """
        TODO: add a name for the model for logging? 
        """

        pwm_left.start(0)
        pwm_right.start(0)

        self.pwm_left = pwm_left
        self.pwm_right = pwm_right
        self.logger = logger 
        self.name = name

            
    def __call__(self, idx, left:int, right:int):
        """
        left: ranged from 0 to 100
        right: ranged from 0 to 100
        """
        self.pwm_left.ChangeDutyCycle(left)
        self.pwm_right.ChangeDutyCycle(right)

        self.logger.log(idx, self.name, dict(left=left, right=right) )

    


class SteeringControl:
    def __init__(self, pwm_left: GPIO.PWM, pwm_right: GPIO.PWM, logger: Logger, name='SteeringControl'):

        
        self.motor_pwm_control = MotorPWMControl(pwm_left, pwm_right, logger)
        self.logger = logger
        self.name = name


    def __call__(self, idx, direction: Literal['left', 'right', 'forward', 'stop']):
        if direction == 'left':
            self.motor_pwm_control(idx, 50, 60)

        elif direction == 'right':
            self.motor_pwm_control(idx, 60, 50)

        elif direction == 'forward':
            self.motor_pwm_control(idx, 60, 60)

        elif direction == 'stop':
            self.motor_pwm_control(idx, 0, 0)
        

        self.logger.log(idx, self.name, {'direction': direction})





def setup_pwm(pin, freq=100) -> GPIO.PWM:
    GPIO.setup(pin, GPIO.OUT)
    return GPIO.PWM(pin, freq) # 100 Hz frequency




In [4]:
GPIO.setmode(GPIO.BOARD)
left = setup_pwm(10,freq=100)
right = setup_pwm(11,freq=100)


/tmp/ipykernel_1775/153077533.py:62: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT)


In [5]:
logger = Logger(save_interval=10, overwrite_ok=True)
steering_control = SteeringControl(left,right, logger)

idx = 0
while True:
    
    logger.log(idx, 'time', {'time': datetime.datetime.now()})
    steering_control(idx, 'forward')

    time.sleep(0.1)
    idx += 1

KeyboardInterrupt: 

In [7]:
right.stop()
left.stop()
GPIO.cleanup()

In [9]:
logger.load_as_df()

/tmp/ipykernel_1775/3395230920.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


value                            \
key            MotorPWMControl           SteeringControl   
idx                                                        
0    {'left': 60, 'right': 60}  {'direction': 'forward'}   
1    {'left': 60, 'right': 60}  {'direction': 'forward'}   
2    {'left': 60, 'right': 60}  {'direction': 'forward'}   
3    {'left': 60, 'right': 60}  {'direction': 'forward'}   
4    {'left': 60, 'right': 60}  {'direction': 'forward'}   
..                         ...                       ...   
65   {'left': 60, 'right': 60}  {'direction': 'forward'}   
66   {'left': 60, 'right': 60}  {'direction': 'forward'}   
67   {'left': 60, 'right': 60}  {'direction': 'forward'}   
68   {'left': 60, 'right': 60}  {'direction': 'forward'}   
69   {'left': 60, 'right': 60}  {'direction': 'forward'}   

                                           
key                                  time  
idx                                        
0    {'time': 2024-02-13 20:48:05.431595}  
1    {'time': 2024-02-13 20:48:05.531785}  
2    {'time': 2024-02-13 20:48:05.631954}  
3    {'time': 2024-02-13 20:48:05.732093}  
4    {'time': 2024-02-13 20:48:05.832765}  
..                                    ...  
65   {'time': 2024-02-13 20:48:11.954887}  
66   {'time': 2024-02-13 20:48:12.055016}  
67   {'time': 2024-02-13 20:48:12.155535}  
68   {'time': 2024-02-13 20:48:12.255691}  
69   {'time': 2024-02-13 20:48:12.355830}  

[70 rows x 3 columns]